# NBA Rotation Code
## Dan McDonough
## May 15th, 2023

This notebook (and the accompanying python file) allows the user to see rotation data for the NBA.

The user can specify the portion of the season to focus on (regular season vs. playoffs, or specific data parameters), and a required frequency for how often a player needs to play a particular minute of the game to be counted as "on the court" during that minute, on average.

In [1]:
import pandas as pd
from nba_api.stats.endpoints import gamerotation, teamgamelog
import plotly.express as px
import pandas as pd
from nba_api.stats.static import teams
import time
import math
from datetime import datetime
from utility import get_game_logs, get_rotation_data, process_rotations, get_player_averages, plot_averages

The first step is to identif which games we want to analyze. I am going to look at the regular season data for the Knicks after Tom Thibodeau shortened the rotation in December

In [2]:
game_log_df = get_game_logs(team='Knicks',season='2022-23',season_mode='Regular Season',start='2023-2-18',end='2023-4-10')
game_log_df.head()

,Team_ID,Game_ID,GAME_DATE,MATCHUP,WL,W,L,W_PCT,MIN,FGM,...,DREB,REB,AST,STL,BLK,TOV,PF,PTS,keep_ind,home_ind
0,1610612752,0022201220,2023-04-09,NYK vs. IND,L,47,35,0.573,240,52,...,34,53,29,8,8,15,24,136,1,1
1,1610612752,0022201209,2023-04-07,NYK @ NOP,L,47,34,0.580,240,38,...,30,37,19,5,2,9,17,105,1,0
2,1610612752,0022201190,2023-04-05,NYK @ IND,W,47,33,0.588,240,52,...,32,52,34,13,8,14,21,138,1,0
3,1610612752,0022201168,2023-04-02,NYK vs. WAS,W,46,33,0.582,240,46,...,32,39,31,12,4,11,17,118,1,1
4,1610612752,0022201154,2023-03-31,NYK @ CLE,W,45,33,0.577,240,49,...,32,48,24,6,5,7,12,130,1,0


Next, I will call the NBA API to get full rotation data for these games

In [3]:
full_rotation_data = get_rotation_data(game_log_df)
full_rotation_data.head()

,GAME_ID,TEAM_ID,TEAM_CITY,TEAM_NAME,PERSON_ID,PLAYER_FIRST,PLAYER_LAST,IN_TIME_REAL,OUT_TIME_REAL,PLAYER_PTS,PT_DIFF,USG_PCT,GAME_DATE,Name
0,0022201220,1610612752,New York,Knicks,203095,Evan,Fournier,5670.0,10220.0,0,-4.0,0.091,2023-04-09,Evan Fournier
1,0022201220,1610612752,New York,Knicks,203095,Evan,Fournier,21600.0,24080.0,0,-12.0,0.375,2023-04-09,Evan Fournier
2,0022201220,1610612752,New York,Knicks,1628392,Isaiah,Hartenstein,1660.0,7820.0,4,1.0,0.103,2023-04-09,Isaiah Hartenstein
3,0022201220,1610612752,New York,Knicks,1628392,Isaiah,Hartenstein,19280.0,25610.0,4,-24.0,0.167,2023-04-09,Isaiah Hartenstein
4,0022201220,1610612752,New York,Knicks,1628404,Josh,Hart,4530.0,11590.0,8,-3.0,0.200,2023-04-09,Josh Hart


I will then transform this data into a dataframe that shows which of the 48 minutes each player played each game

In [4]:
full_player_data = process_rotations(full_rotation_data)
full_player_data.head()

,Minute,Name,On
0,0,Evan Fournier,0
1,1,Evan Fournier,0
2,2,Evan Fournier,0
3,3,Evan Fournier,0
4,4,Evan Fournier,0


Finally, I will calculate the average subtitution patterns of each player

In [11]:
average_player_data = get_player_averages(full_player_data, game_log_df, 0.5)
average_player_data.head()

,Name,Start,End,Delta
0,Immanuel Quickley,0,47,47
1,Isaiah Hartenstein,7,18,11
2,Isaiah Hartenstein,32,45,13
3,Jalen Brunson,0,10,10
4,Jalen Brunson,17,33,16


Here is a visual output of the data

In [12]:
plot_averages(average_player_data)